# Dispersal in metacommunities

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import ipywidgets as widgets
%matplotlib widget

In [1]:
def moran_dispersal(n_indiv, n_years, m_patches, init_f1, dispersal):
    """ Implements the Moran model including constant selection and simulates it in time. 
        
        Parameters
        ----------
        n_indiv: int
            number of individuals in the community
        n_years: int
            number of potential 'turnovers' of the community
        m_patches: int
            number of separate patches linked by dispersal
        init_f1: float
            initial frequency of species 1
        avg_fit_ratio: float
            fitness ratio of species 1 and 2 when both are in equal frequency
        freq_dep: float
            Strength of frequency dependence for selection
        dispersal: float
            probability that a new birth is drawn from metacommunity
            
        Return
        ------
        moran: np.array (n_years, 2)
            contains the species frequencies for each year of the simulation """

    # For this we're going to track only species 1 and instead of the columns representing
    # different species they will be different patches
    moran = np.zeros((n_years, m_patches))
    moran[0] = np.array(init_f1)

    # Now our community vector is instead a metacommunity matrix
    meta = np.random.choice([0, 1], size=(m_patches, n_indiv), replace=True, p=moran[0])

    # now we can do the loop as in vellend. I'm going to write it a bit differently
    # but the idea is basically the same. (yes its slower)
    for year in range(1, n_years):

        # we now need to check each patch as well as each year
        for patch in range(m_patches):

            # for each year in each patch we do replacement for every individual
            for indiv in range(n_indiv):

                # now every time we replace an individual we need to check whether or not
                # dispersal occurs. Draw from entire metacommunity
                if np.random.uniform() < dispersal:
                    pr1 = np.sum(meta == 1) / (n_indiv * m_patches)
                # or draw from local community
                else:    
                    pr1 = np.sum(meta[patch] == 0) / n_indiv

                # replace one individual with another
                death = np.random.randint(n_indiv)
                birth = np.random.choice([0, 1], p=[pr1, 1 - pr1])
                meta[patch, death] = birth

            # when we're done looping over all of the individuals we can update
            # frequencies for the year (the timescale we care about)
            f1 = np.sum(meta[patch] == 0) / n_indiv
            moran[year, patch] = f1

    return moran

In [ ]:
def draw_simulation(iterations, individuals, years, patches, initial_freq, dispersal):
    # the plot bit, this just makes a blank figure
    fig, ax = plt.subplots(ncols=3, figsize=(10,4), gridspec_kw = {'width_ratios':[2, 3, 1]})
    # plot the frequency dependence as in Ch. 6
    # frequency range and neutral line as above
    f1 = np.arange(0, 1, 0.01)
    ax[0].axhline(1, linestyle='--', color='grey', alpha=0.4)
    # dependence curve
    ax[0].plot(f1, avg_fit_ratio * np.exp(freq_dep*(f1 - 0.5)))
    ax[0].set_xlabel('Frequency of species 1')
    ax[0].set_ylabel('Fitness ratio 1:2')

    # trajectory labels
    ax[1].set_xlabel('Years')
    ax[1].set_ylabel('Species 1 frequency')
    ax[1].set_ylim([0, 1])
    # distribution labels
    ax[2].set_xlabel('Count')
    ax[2].set_xlim([0, iterations])
    ax[2].set_ylim([0, 1])

    # we're going to plot a distribution too need an array for it
    dist = np.zeros(iterations)

    # we run the simulation and draw a trajectory
    for i in tqdm(range(iterations)):
        simulation = moran_dispersal(individuals, years, patches, initial_freq, dispersal)
        ax[1].plot(range(years), simulation[:, 0])
        # add final freq to our dist array
        dist[i] = simulation[-1, 0]

    # plot the distribution too
    ax[2].hist(dist, orientation='horizontal')
    ax[2].axhline(np.mean(dist), linestyle='--')


    plt.tight_layout()    
    plt.show()

# set up the interface
widgets.interact_manual(draw_simulation, iterations=(5, 50, 5), individuals=(10, 1000, 10), years=(5, 200, 5),
                  patches=(1, 10, 1), initial_freq=(0.0, 1.0, 0.01), dispersal=(0, 1.0, 0.01))